In [ ]:
# initialize hyperparameters

In [15]:
# initialize parameters

def initialize_parameters(parameters, layer, dimension):
    
    W = tf.get_variable("W" + layer, dimension["W"], initializer = tf.contrib.layers.xavier_initializer(seed = 1)); # Remove after
    b = tf.get_variable("b" + layer, dimension["b"], initializer = tf.zeros_initializer())
    
    parameters ={
        "W" + layer: W
        , "b" + layer: b
    };
    
    return parameters;

In [16]:
# forward propogation

def forward_propogation(X, parameters, hyperparameters, keep_prob = 1):
    
    A[0] = X;
    
    layers = hyperparameters["layers"];
    
    for i in range(len(layers)):
        W[i+1] = parameters["W" + (i+1)];
        b[i+1] = parameters["b" + (i+1)];
        A[i+1] = tf.nn.relu(tf.matmul(A[i], W[i+1]) + b[i+1]);
        A[i+1] = tf.nn.dropout(A[i + 1], keep_prob);
    
    
    return A[len(layers)];

In [17]:
# model

def model (X_train, Y_train, X_test, Y_test, hyperparameters):
    
    layers = hyperparameters["layers"];
    learning_rate = hyperparameters["learning_rate"];
    epochs = hyperparameters["epochs"];
    minibatch_size = hyperparameters["minibatch_size"];
    
    # initialize weight and biases
    parameters = {}
    for i in range(len(layers) - 1):
        parameters = initialize_parameters(parameters, i + 1, {"W": [layers[i+1], layers[i]], "b": [layers[i+1], 1]});
    
    # Forward Propogation
    logits = forward_propogation(X, parameters, hyperparameters);
    
    cost = tf.reduce_mean(
        # Maybe need transpose ?
        tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels)
        # Need regulurazation
    );
    
    # ...izer :-)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost);
    initializer = tf.global_variables_initializer();
    
    # Start session
    with tf.Session() as sess:
        # Run the initialization
        sess.run(initializer)
        
        # Do the training loop
        for epoch in range(epochs):
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size)
            
            for minibatch in minibatches:
                
                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict = {X: minibatch_X, Y: minibatch_Y, keep_prob: 0.2})
        
    
    
    
    return parameters;

In [9]:
# hyperparameters searching

class tuning():
    def set_learning_rate(learning_rate):
        learning_rate = learning_rate;
        return None;
    
    def set_layer(layer_index, neurons):
        layers;
        return None;
    
    def set_mini_batch_size():
        
        return None;
    
    return None;


4
4
4
